In [133]:
import pandas as pd 
import string
import numpy as np

df = pd.read_csv('fp2.csv')


def clean(col1, col2):
    df[col1] = df[col2].astype(str) + df[col1].astype(str)
    df[col1] = [w.replace('nan', '').strip('[]\'').replace("'", '').replace(', ', ';') for w in df[col1]]

for c1, c2 in zip(['Title', 'Year', 'Publisher', 'Genre'], ['Title:', 'Release Date:', 'Publisher:', 'Genre:']):
    clean(c1, c2)
    
df['Year'] = [y[-4:] if len(y) > 4 else y for y in df['Year']]
df['No_Players'] = [str(w).strip(string.ascii_letters+' ') if isinstance(w, str) else w for w in df['No_Players']]
df['No_Players'] = [entry.split('-')[1] if isinstance(entry, str) and '-' in entry else entry for entry in df['No_Players']]        
df = df[df['Percentage of Positive Reviews'].notna()]
df['Franchise:'] = [w.strip('[]\'') if isinstance(w, str) else w for w in df['Franchise:']]
df['Developer:'] = [w.strip('[]\'') if isinstance(w, str) else w for w in df['Developer:']]
df['Genre'] = [w.replace(']', ';') for w in df['Genre']]
df = df.drop(['Title:', 'Release Date:', 'Publisher:', 'Genre:', 'Publisher', 'Developer:', 'Discounted Price', 'Unnamed: 0'], axis=1)
df = df.dropna(thresh=6)

def x(s):
    if s != 0:
        return 1
    else:
        return 0

df['Franchise:'] = df['Franchise:'].fillna(0)
df['Franchise:'] = df['Franchise:'].map(x)
df['Percentage of Positive Reviews'] = [w.strip('%') if isinstance(w, str) else w for w in df['Percentage of Positive Reviews']]
df['Price'] = [w.strip('$') if isinstance(w, str) else w for w in df['Price']]
df

# remove percentage and dollar sign, add genre columns

,Title,Year,Genre,Platform,Metascore,Avg_Userscore,No_Players,Number of Ratings,Percentage of Positive Reviews,Price,Franchise:
13,GrandTheftAutoV]Grand Theft Auto V,2014,Action;Adventure;Action Adventure;Modern;Open-...,XboxOne,97,7.8,30,773837.0,79,29.99,1
14,GrandTheftAutoV]Grand Theft Auto V,2013,Action;Adventure;Modern;Action Adventure;Moder...,PlayStation3,97,8.3,16,773837.0,79,29.99,1
15,GrandTheftAutoV]Grand Theft Auto V,2013,Action;Adventure;Modern;Action Adventure;Moder...,Xbox360,97,8.3,16,773837.0,79,29.99,1
16,GrandTheftAutoV]Grand Theft Auto V,2014,Action;Adventure;Action Adventure;Modern;Open-...,PlayStation4,97,8.3,30,773837.0,79,29.99,1
17,GrandTheftAutoV]Grand Theft Auto V,2015,Action;Adventure;Modern;Action Adventure;Open-...,PC,96,7.7,32,773837.0,79,29.99,1
...,...,...,...,...,...,...,...,...,...,...,...
5847,KingdomUnderFire2,2019,Action;Massively Multiplayer;RPG;Strategy,NaN,NaN,NaN,NaN,1399.0,73,9.99,0
5848,Thief,2014,Action;Adventure,NaN,NaN,NaN,NaN,11296.0,72,19.99,1
5849,STARWARSTheForceUnleashedUltimateSithEdition,2009,Action,NaN,NaN,NaN,NaN,3805.0,69,19.99,1
5850,BloodBowl2LegendaryEdition,2017,Sports;Strategy;Violent;Gore,NaN,NaN,NaN,NaN,0.0,No Rating,29.99,1


In [94]:
s = df['Genre']
s = [entry.split(';') for  entry in s.values]
s



[['Action', 'Adventure', 'Action Adventure', 'Modern', 'Open-World'],
 ['Action', 'Adventure', 'Modern', 'Action Adventure', 'Modern', 'Open-World'],
 ['Action', 'Adventure', 'Modern', 'Action Adventure', 'Modern', 'Open-World'],
 ['Action', 'Adventure', 'Action Adventure', 'Modern', 'Open-World'],
 ['Action', 'Adventure', 'Modern', 'Action Adventure', 'Open-World'],
 ['RPG', 'Role-Playing', 'Action RPG', 'Action RPG'],
 ['RPG', 'Role-Playing', 'Action RPG', 'Action RPG'],
 ['RPG', 'Role-Playing', 'Action RPG', 'Action RPG'],
 ['Action',
  'Adventure',
  'Action',
  'Shooter',
  'Shooter',
  'First-Person',
  'Sci-Fi',
  'Sci-Fi',
  'Arcade'],
 ['Action',
  'Adventure',
  'Action',
  'Shooter',
  'Shooter',
  'First-Person',
  'Sci-Fi',
  'Sci-Fi',
  'Arcade'],
 ['Action',
  'Adventure',
  'Action',
  'Shooter',
  'Shooter',
  'First-Person',
  'Sci-Fi',
  'Sci-Fi',
  'Arcade'],
 ['Action',
  'Action',
  'Shooter',
  'Shooter',
  'First-Person',
  'Sci-Fi',
  'Sci-Fi',
  'Arcade'],
 ['

In [72]:

def clean_up(column): 
    
    new_df = df[column].values.reshape(-1,1)
    new_col = []
    
    for item in new_df:
        if type(item[0]) == str:
            
        
            col_data = item[0].split(',')
            column_lst = []
       
            for i in range(len(col_data)):
                col_data[i] = col_data[i].strip("'][ '")
                column_lst.append(col_data[i])
                
        
            new_col.append(column_lst)
    
        else: 
        
            new_col.append([])
        
        
    return new_col

In [75]:
cleanable_columns = ['Genre']

for item in cleanable_columns:
    clean_column = clean_up(item)
    
    df = df.drop(item, axis = 1)
    new_series = pd.Series(clean_column)
    df[item] = new_series

# df = df.drop('Unnamed: 0', axis = 1)


In [109]:
def count_unique(series):
    
    unique_items = []
    
    for item in series:
        for i in range(len(item)):
            
            if item[i] not in unique_items:
                
                unique_items.append(item[i])
#     unique_items.remove('')
    unique_items = {item:i for i, item in enumerate(unique_items)}
    return unique_items
            
            
        
    

In [107]:
import numpy as np
def one_hot(series, unique_lst):
    encoded = np.zeros((len(series), len(unique_lst)))
    for i, genres in enumerate(series):
        encoded[i][[unique_lst[g] for g in genres]] = 1
        
    return encoded



In [134]:
unique = count_unique(s)
unique
genres = one_hot(s, unique)
for g in unique.keys():
    df[g] = genres[:, unique[g]]
df = df.drop('Genre', axis=1)
df

,Title,Year,Platform,Metascore,Avg_Userscore,No_Players,Number of Ratings,Percentage of Positive Reviews,Price,Franchise:,...,Animation & Modeling,Design & Illustration,Utilities,,Web Publishing,Video Production,Game Development,Audio Production,Violent,Gore
13,GrandTheftAutoV]Grand Theft Auto V,2014,XboxOne,97,7.8,30,773837.0,79,29.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,GrandTheftAutoV]Grand Theft Auto V,2013,PlayStation3,97,8.3,16,773837.0,79,29.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,GrandTheftAutoV]Grand Theft Auto V,2013,Xbox360,97,8.3,16,773837.0,79,29.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,GrandTheftAutoV]Grand Theft Auto V,2014,PlayStation4,97,8.3,30,773837.0,79,29.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,GrandTheftAutoV]Grand Theft Auto V,2015,PC,96,7.7,32,773837.0,79,29.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5847,KingdomUnderFire2,2019,NaN,NaN,NaN,NaN,1399.0,73,9.99,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5848,Thief,2014,NaN,NaN,NaN,NaN,11296.0,72,19.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5849,STARWARSTheForceUnleashedUltimateSithEdition,2009,NaN,NaN,NaN,NaN,3805.0,69,19.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5850,BloodBowl2LegendaryEdition,2017,NaN,NaN,NaN,NaN,0.0,No Rating,29.99,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [80]:
for item in cleanable_columns: 
    
    unique = count_unique(item)

    one_hot(item, unique)
    df = df = df.drop(item, axis = 1)
    print(item)

AttributeError: 'list' object has no attribute 'split'

In [ ]:
df

In [ ]:
df.to_csv("new_data")